# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [24]:
!pip install selenium


In [25]:
from selenium import webdriver #permite criar navegador
from selenium.webdriver.common.keys import Keys #permite escrever no navegador
from selenium.webdriver.common.by import By #permite selecionar itens no navegador

#abrir navegador
navegador = webdriver.Chrome()

#entrar no google
navegador.get('https://google.com')

#pesquisar no google por cotação dólar
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath',
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)


#pegar cotação do euro
#entrar no google
navegador.get('https://google.com')

#pesquisar no google por cotação dólar
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath',
'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)


#pegar cotação do ouro

navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.')
print(cotacao_ouro)


navegador.quit() #fecha o navegador

5.2222
5.111724359
288.92


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [26]:
#atualizar a base de preços
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [27]:
#atualizar coluna de cotação
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

display(tabela)
#atualizar coluna de preço de compra  = preço original * cotação
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

#atualizar coluna de preço de venda = preço de compra *  margem
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.222200,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.111724,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.222200,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.222200,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.111724,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.222200,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,288.920000,7000.00,1.15,8050.000


In [28]:
#exportar a base
tabela.to_excel('produtos_novo.xlsx', index = False)